Nuha BinTayyash, 2020

This notebook shows the ROC and precision-recall curves resulted from running GPcounts with one sample test on simulated bulk RNA-seq datasets.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics
import pandas as pd

In [3]:
DESeq2 = pd.read_csv('alpha_DESeq2.csv',index_col=[0])
DESeq2 = DESeq2.dropna()
print(DESeq2.shape)
GPcounts_NB = pd.read_csv('ll_Negative_binomial_normalized_alpha_counts.csv',index_col=[0])
GPcounts_NB = GPcounts_NB.loc[DESeq2.index]
print(GPcounts_NB.shape)

(15859, 6)
(15859, 3)


In [4]:
DESeq_sorted = DESeq2.sort_values(by=['padj'])
DESeq_sorted

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ENSMUSG00000000394,338382.020415,2.901758,0.092755,830.126895,1.520832e-182,2.411888e-178
ENSMUSG00000023944,1340.911792,-1.971445,0.085539,590.292141,2.164533e-130,1.716366e-126
ENSMUSG00000040856,21.339180,-11.555445,0.489032,426.134934,1.127530e-94,5.960498e-91
ENSMUSG00000021270,381.298312,-2.550084,0.136911,387.655126,2.681409e-86,1.063112e-82
ENSMUSG00000071178,65.258012,-12.370333,0.622465,350.013766,4.208641e-78,1.334897e-74
...,...,...,...,...,...,...
ENSMUSG00000081470,0.295732,-0.299563,1.108374,-0.078420,1.000000e+00,1.000000e+00
ENSMUSG00000039477,8.119364,-0.016673,0.778194,0.000505,9.820720e-01,1.000000e+00
ENSMUSG00000030321,2.660504,-0.042777,1.873896,0.000527,9.816774e-01,1.000000e+00
ENSMUSG00000039963,6.018876,-0.001209,1.181372,0.000008,9.977488e-01,1.000000e+00


In [5]:
GPcounts_NB_sorted = GPcounts_NB.sort_values(by=['log_likelihood_ratio'], ascending=False)
GPcounts_NB_sorted

,Dynamic_model_log_likelihood,Constant_model_log_likelihood,log_likelihood_ratio
ENSMUSG00000000394,-4234.824594,-4689.430336,454.605742
ENSMUSG00000023944,-2420.437926,-2722.764490,302.326565
ENSMUSG00000040856,-775.121202,-1070.490885,295.369683
ENSMUSG00000062510,-542.096145,-804.966877,262.870732
ENSMUSG00000066366,-461.263969,-681.705865,220.441896
...,...,...,...
ENSMUSG00000071281,-1127.428930,-1122.351385,-5.077546
ENSMUSG00000021453,-1397.160637,-1391.894499,-5.266138
ENSMUSG00000029833,-1112.240541,-1106.753560,-5.486981
ENSMUSG00000060477,-758.312815,-751.734730,-6.578085


In [6]:
DE_alpha = pd.read_csv('Sheet 2_Genes highly expressed in α-lineage.csv',index_col=[0])
DE_alpha.head()

,Symbol,Description,Transcription Factor,log2FoldChange,Adjusted p-value
ID,,,,,
ENSMUSG00000000394,Gcg,glucagon,0,-11.282196,0.0
ENSMUSG00000001504,Irx2,Iroquois related homeobox 2 (Drosophila),1,-9.326386,0.0
ENSMUSG00000004631,Sgce,"sarcoglycan, epsilon",0,-9.841030,0.0
ENSMUSG00000027524,Edn3,endothelin 3,NaN,-9.563078,0.0
ENSMUSG00000027971,Ndst4,N-deacetylase/N-sulfotransferase (heparin gluc...,NaN,-7.362420,0.0


In [40]:
DESeq_index = list(DESeq_sorted.index.values)
best_DE_alpha_index = list(DE_alpha.index.values)
GPcounts_NB_index = list(GPcounts_NB_sorted.index)
DESeq_idxes = [idx for idx, val in enumerate(DESeq_index) if val in best_DE_alpha_index]
GPcounts_idxes = [idx for idx, val in enumerate(GPcounts_NB_index) if val in best_DE_alpha_index]
#GPcounts_DESeq_index = [idx for idx, val in enumerate(DESeq_index) if val in GPcounts_NB_index]
DE_alpha_idxes  = list(range(0,len(best_DE_alpha_index)))
DESeq_2_idxes = list(range(0,len(DESeq_index)))
GPcounts_idxes_2 = []
for i in GPcounts_NB_index:
    GPcounts_idxes_2.append(DESeq_index.index(i))

In [27]:
from scipy import stats

In [28]:
rho, pval = stats.spearmanr(DESeq_idxes,DE_alpha_idxes)
print(rho,pval)

1.0 0.0


In [12]:
rho, pval = stats.spearmanr(DE_alpha_idxes,GPcounts_idxes)
print(rho,pval)

1.0 0.0


In [13]:
rho, pval = stats.spearmanr(DESeq_idxes,GPcounts_idxes)
print(rho,pval)

1.0 0.0


In [43]:
rho, pval = stats.spearmanr(DESeq_2_idxes,GPcounts_idxes_2)
print(rho,pval)

0.7682108191538324 0.0


In [44]:
DESeq_2_idxes

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [45]:
GPcounts_idxes_2

[0,
 1,
 2,
 2073,
 10,
 3,
 6,
 6695,
 15,
 5,
 4,
 46,
 4955,
 7,
 32,
 8,
 13825,
 9,
 11,
 874,
 12,
 14,
 13,
 6503,
 20,
 16,
 3343,
 5688,
 21,
 17,
 2920,
 2822,
 3744,
 18,
 191,
 127,
 29,
 63,
 22,
 31,
 19,
 4993,
 24,
 23,
 89,
 26,
 25,
 3120,
 13997,
 6627,
 27,
 36,
 125,
 37,
 28,
 30,
 33,
 34,
 56,
 35,
 49,
 72,
 38,
 3121,
 41,
 276,
 40,
 39,
 220,
 50,
 43,
 8217,
 44,
 42,
 170,
 332,
 71,
 48,
 122,
 47,
 55,
 1432,
 81,
 45,
 695,
 59,
 77,
 51,
 65,
 92,
 52,
 66,
 53,
 80,
 57,
 58,
 13438,
 54,
 645,
 61,
 64,
 70,
 15038,
 471,
 153,
 67,
 60,
 1963,
 69,
 1074,
 68,
 76,
 185,
 94,
 4173,
 62,
 4412,
 5201,
 173,
 6612,
 115,
 4142,
 74,
 349,
 704,
 73,
 330,
 1614,
 78,
 75,
 4123,
 257,
 15425,
 98,
 79,
 200,
 160,
 87,
 573,
 7024,
 647,
 85,
 93,
 99,
 193,
 119,
 672,
 84,
 163,
 1860,
 101,
 118,
 14408,
 130,
 161,
 82,
 86,
 88,
 83,
 1058,
 157,
 955,
 90,
 1167,
 776,
 121,
 225,
 105,
 112,
 6757,
 9371,
 3101,
 1503,
 95,
 126,
 144,
 140,
 